Vamos explorar um pouco os principais topicos de apresentados nos comentarios de cada usuario.

Para esse fim poderiamos utilizar uma abordagem baseada em LLM como na Identificação de Promotor e Detrator.\
Mas na intenção de demonstrar um maior leque de conhecimentos e embassamento classico, vamos utilizar BERTopic\
Que demanda do conhecimento de preprocessamento e limpeza classicos de NLP

In [3]:
import pandas as pd

df = pd.read_csv('data/04_nlp_classificados.csv')

In [4]:
import re
import unidecode
import nltk
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer

# Baixar stopwords e stemmer do NLTK (roda só uma vez)
nltk.download("stopwords")
nltk.download('rslp')

# Lista de stopwords em português
stop_words = set(stopwords.words("portuguese"))

stemmer = RSLPStemmer()

# Função de pré-processamento
def preprocess(text):
    text = unidecode.unidecode(text)
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"\d+", "", text)
    tokens = text.split()
    tokens = [
        stemmer.stem(t) 
        for t in tokens 
        if t not in stop_words and len(t) > 4
    ]
    return " ".join(tokens)

# Pré-processar os comentários
df["comments_pre"] = df["comments"].astype(str).apply(preprocess)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/phelipe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /home/phelipe/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


Acima usamos nltk para pegar uma lista pronta de stopwords\
(que descrevem palavras sem conteudo proprio, apenas apoiam a coerencia do texto).

In [5]:

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer

# Embedding model (CPU)
embedding_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2", device="cpu")

# Criar modelo BERTopic
topic_model = BERTopic(embedding_model=embedding_model, 
                        language="portuguese",
                        min_topic_size=6)

# Gerar tópicos
topics, probs = topic_model.fit_transform(df["comments_pre"])

# Adicionar ao DataFrame
df["topic"] = topics

df.drop(columns=["comments_pre"], inplace=True)

# Mostrar frequência dos tópicos
print(df["topic"].value_counts())

/home/phelipe/projects/pessoal/churn_basedOn_nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


topic
 0    58
 1    15
 2    11
 3    10
-1     8
Name: count, dtype: int64


Foram encontrados 6 topicos (e 1 topico nulo) ao longo dos comentarios

In [6]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,8,-1_pixel_contat_entr_problem,"[pixel, contat, entr, problem, resolv, imag, f...",[problem fatur dest val cobr incorret consig e...
1,0,58,0_internet_serv_satisfeit_conexa,"[internet, serv, satisfeit, conexa, problem, n...",[satisfeit serv telefon movel cham cl internet...
2,1,15,1_empr_complet_dess_telefon,"[empr, complet, dess, telefon, horri, experien...",[telefon movel dess empr verd desastr cham com...
3,2,11,2_qual_nit_canal_excel,"[qual, nit, canal, excel, imag, pouc, mens, va...",[bast satisfeit serv qual canal excel imag nit...
4,3,10,3_canal_sinal_const_client,"[canal, sinal, const, client, pess, atend, alg...",[serv dess empr pi maior canal func sinal cons...


O BERTopic apresenta alguns nomes baseado nas palavras chaves, mas para facilitar o entendimento podemos renomear:

In [7]:
map_topic = {
    -1:"other",
    0:"internet_connection",
    1:"critical_appointments",
    2:"quality_tv_channels",
    3:"signal"
    }

df["topic"] = df["topic"].map(map_topic)

Em analise ocular encontrei mais um topico que acabou não sendo descrito pelo BERTopic: Atendimento

vamos adiciona-lo manualmente

In [8]:
# Definir tópico 'atendimento' onde 'atendimento' aparecer nos comentários
df.loc[df['comments'].str.contains('atendimento', case=False, na=False), 'topic'] = 'atendimento'

Aqui de forma bem simples, pra todo comentario que tiver a palavra 'atendimento' assumimos que o assunto é 'atendimento'

Explorei usar o lemma 'atend' mas muitos comentarios utilizam expressões como 'o serviço tem atendido minhas expectativas'

In [9]:
df.to_csv("data/05_nlp_classificados_topicos.csv", index=False)

Os topicos que conseguimos parece sempre ter uma relação muito grande com o serviço, vamos investigar essa relação para entender a utilidade desses topicos

In [10]:
import pandas as pd

df = pd.read_csv("data/05_nlp_classificados_topicos.csv")

df.head(2)

,id,name,age,gender,location,marital_status,qtd_dependents,start_date,churn_date,price,service,tma,data_volume,qtd_appointments,comments,churn,classificacao,topic
0,1,João Silva,35,Masculino,SP,Casado,2,2018-05-10,NaN,79.99,Telefonia Móvel,270.0,2.3,2.0,O serviço de internet tem sido instável. Estou...,False,Detrator,internet_connection
1,2,Maria Santos,28,Feminino,RJ,Solteiro,0,2019-03-15,NaN,109.99,Internet,0.0,0.0,0.0,Estou satisfeita com o serviço. A velocidade d...,False,Promotor,internet_connection


In [11]:
for service in df["service"].unique():
    print(service)
    for topic in df["topic"].unique():
        temp_df = df[(df["service"]==service) & (df["topic"]==topic)]
        print(f"    {topic}: {temp_df.shape[0]}")


Telefonia Móvel
    internet_connection: 13
    atendimento: 0
    other: 2
    signal: 0
    critical_appointments: 8
    quality_tv_channels: 0
Internet
    internet_connection: 30
    atendimento: 1
    other: 0
    signal: 0
    critical_appointments: 0
    quality_tv_channels: 0
Telefonia Fixa
    internet_connection: 7
    atendimento: 9
    other: 0
    signal: 0
    critical_appointments: 7
    quality_tv_channels: 0
TV a Cabo
    internet_connection: 1
    atendimento: 7
    other: 4
    signal: 2
    critical_appointments: 0
    quality_tv_channels: 11


Basicamente com exceção de critical_appointments e atendimento, cada outro topico é bem associado a um serviço

Vamos explorar especificamente os detratores

In [12]:
df_detratores = df[df["classificacao"]=="Detrator"]

for service in df_detratores["service"].unique():
    print(service)
    for topic in df_detratores["topic"].unique():
        temp_df = df_detratores[(df_detratores["service"]==service) & (df_detratores["topic"]==topic)]
        print(f"    {topic}: {temp_df.shape[0]}")


Telefonia Móvel
    internet_connection: 3
    atendimento: 0
    other: 0
    signal: 0
    critical_appointments: 8
Internet
    internet_connection: 14
    atendimento: 1
    other: 0
    signal: 0
    critical_appointments: 0
TV a Cabo
    internet_connection: 0
    atendimento: 7
    other: 4
    signal: 2
    critical_appointments: 0
Telefonia Fixa
    internet_connection: 0
    atendimento: 2
    other: 0
    signal: 0
    critical_appointments: 6


Certo, aqui podemos começar a ter insights sobre onde atuar:

**critical_appointments** no geral tem pouca informação, é apenas alguem bem chateado com os serviços insultando a empresa.\
Talvez possamos encontrar nesses casos reclamações anteriores que possam indicar melhor as dores do cliente e possiveis caminhos para reverter esse, provavel, churn eminente

Logo depois tempos **Atendimento**, que chama bastante atençao por ser um topico cross os serviços.

E então temos as reclamações relacionadas ao serviço em si: **internet_connection** e **signal**

Vamos explorar uma ultima ótica deste ultimos 2 topicos no proximo IPYNB